In [1]:
# pip install transformers torch datasets evaluate rouge-score sentencepiece accelerate pandas matplotlib

In [2]:
from transformers import (
    pipeline,                          # 고수준 API - 가장 쉬운 방법
    AutoTokenizer,                     # 자동 토크나이저
    AutoModelForSeq2SeqLM,            # 자동 모델 로더 (Seq2Seq 작업용)
    T5TokenizerFast,                  # T5 전용 고속 토크나이저
    T5ForConditionalGeneration,       # T5 모델 클래스
    BartForConditionalGeneration,     # BART 모델 클래스
    PreTrainedTokenizerFast,          # 사전학습 토크나이저 기본 클래스
    DataCollatorForSeq2Seq,           # Seq2Seq 학습용 데이터 콜레이터
    Seq2SeqTrainingArguments,         # Seq2Seq 학습 하이퍼파라미터
    Seq2SeqTrainer,                   # Seq2Seq 전용 트레이너
)

c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 버전확인
import torch 
import transformers
print(torch.__version__)
print(transformers.__version__)
#

2.9.1+cpu
4.57.1


In [4]:
# pipline을 이용한 간단한 문서 요약
'''
1. 모델을 다운로드 및 로딩
2. 토크나이제이션(문자 -> 숫자)
3. 모델추론(요약생성)
4. 디코딩(숫자->문자)

장점 : 적은 코드로 실행가능(2~3줄)
단점 : 세밀한 제어 어려움

언제 : 
- 빠른 프로토 타입 제작
- 간단한  데모
- 성능테스트
'''
summarizer = pipeline("summarization", 
                      model="facebook/bart-large-cnn",
                      device=0 if torch.cuda.is_available() else -1
                      )

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
summary_result = summarizer(ARTICLE,
                            max_length=130,
                            min_length=30,
                            do_sample=False
                            )
print(summary_result[0]['summary_text'])

Device set to use cpu


Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.


In [5]:
summary_text = summary_result[0]['summary_text']
# 통계분석
compression_ratio = len(summary_text) / len(ARTICLE) * 100
word_reduced = len(ARTICLE.split()) - len(summary_text.split())
print(f'원문 길이 : {len(ARTICLE)} 문자 ({len(ARTICLE.split())} 단어)')
print(f'요약문 길이 : {len(summary_text)} 문자 ({len(summary_text.split())} 단어)')
print(f'압축률 : {compression_ratio:.2f}%')
print(f'단어 감소 : {word_reduced} 단어')

원문 길이 : 2298 문자 (369 단어)
요약문 길이 : 262 문자 (47 단어)
압축률 : 11.40%
단어 감소 : 322 단어


In [7]:
# T5 모델과 AutoModel을 이용한 문서 요약
# 모든 nlp작업을 텍스트 -> 텍스트 형식으로 통일
# 요약 : "summarize": [원문] -> [요약문]"
# 번역 : "translate": [원문] -> [번역문]"
# 분류 : "sentiment [원문] -> [클래스]"

# 단점 : Task Prefix 필수(없으면 급격한 성능 저하) -> 위의 예시처럼 문제의 유형을 알려주는 짧은 문장을 입력 앞에 붙힘

MODEL_NAME = "t5-base"  # t5-small, t5-large, t5-3b, t5-11b
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 전처리
# 1. 공백제거
# 2. 줄바꿈을 공백으로 변환(모델은 줄바꿈을 잘 처리 못함)
# 3. Task Prefix 추가 (예: "summarize: ")
preprocess_text = ARTICLE.strip().replace("\n", " ")
input_text = f'summarize: {preprocess_text}'

# 토크나이제이션
tokenized_text = tokenizer.encode(
    input_text,
    return_tensors="pt",
    max_length=512,
    truncation=True
).to(device)

print(f'토큰수 : {tokenized_text.size(1)}')
print(f'텐서형태 : {tokenized_text.shape} (배치크기 =1, 시퀀스 길이 = {tokenized_text.size(1)})')
print(f'첫 10개 토큰ID : {tokenized_text[0][:10].tolist()}')

# 디코딩(숫자 -> 텍스트)
decoded_text = tokenizer.decode(tokenized_text[0][:20], skip_special_tokens=True)
print(f'첫20개토큰 디코딩 : {decoded_text}')

토큰수 : 512
텐서형태 : torch.Size([1, 512]) (배치크기 =1, 시퀀스 길이 = 512)
첫 10개 토큰ID : [21603, 10, 368, 1060, 41, 254, 17235, 61, 10555, 301]
첫20개토큰 디코딩 : summarize: New York (CNN)When Liana Barrientos was 23 years old,


In [9]:
# 요약생성 Beam Search 사용
# 여러 가능성을 동시에 탐색하면서 최적의 요약 찾기
# baam=4, min_length=30, max_length=100
summary_ids = model.generate(
    tokenized_text,
    num_beams=4,
    max_length=100,
    min_length=30,
    early_stopping=True,
    no_repeat_ngram_size=3,
)

# 생성완료
# 디코딩
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f'요약통계')
print(f'원문의 길이 : {len(ARTICLE)} 문자 ({len(ARTICLE.split())} 단어)')
print(f'요약문의 길이 : {len(output)} 문자 ({len(output.split())} 단어)')
print(f'압축률 : {len(output) / len(ARTICLE):.2f}')


요약통계
원문의 길이 : 2298 문자 (369 단어)
요약문의 길이 : 148 문자 (23 단어)
압축률 : 0.06


In [ ]:
# T5 모델
# Task prefix
# 번역
# 